Job requirement:
https://www.upwork.com/ab/f/offer/10061018

Deliverable is csv with all businesses that accept crypto in given target markets with all data points included. 

Target Markets: 
New York City (including surrounding boroughs), NY 
Atlanta Metro , GA 
Chicago, IL 
Washington DC 
Miami, FL 
Philadelphia, PA 
Houston, TX 
Detroit ,MI 
Memphis , TN 
Baltimore , MD 
Los Angeles , CA 
Dallas , TX 
Charlotte, NC 
Jacksonville, FL 
Milwaukee, WI 
Indianapolis, IN 
Columbus, OH 
Cleveland, OH 
New Orleans , LA 
Nashville, TN 
St. Lous, MO 
Birmingham, AL 
Boston, MA 
Newark, NJ 
Jackson, MS 
Kansas City, MO 
Louiville, KY 
Baton Rouge, LA 
Cincinnati , OH 
Raleigh , NC 
Montgomery, AL 
Greensboro, NC 
Oakland, CA 
Shreveport, LA 
Augusta, GA 
Norfolk, VA 
Richmond, VA 
Buffalo, NY 
Mobile, AL 
Durham, NC 
Macon, GA 

Data Points: 
Business Name 
Business Type (restaurant, bakery, etc) please be specific i.e Italian Restaurant 
Location (Address) 
Phone Number 
Website 

Can utilize Coinmap,org plus other web/online resources to compile csv.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import requests
import pandas as pd
import geopy
import numpy as np

# 1. Get all detailed data from Upwork

In [2]:
# # total entry: 14479 (query performed on March 24, 2019), use chunk_size = 1000

# dfs = []
# for c in range(0, 15000, 1000):
#     print(c)
#     url='https://coinmap.org/api/v1/venues/?offset={}&limit=1000&mode=full'.format(c)
#     response = requests.get(url)
#     print(response)
#     try:
#         data = response.json()['venues']
#         df = pd.DataFrame(data)
#         dfs.append(df)
#     except:
#         pass

# data = pd.concat(dfs, ignore_index=True)

# # note: problem with 6511, 6528, 10112, 10113, 13476, 13484, 13791-13795, 13797-13816, response showing a "500: Internal Server Error"
# # total valid entries: 14479-6-5-20=14479-31=14448 

In [3]:
# update category column to lower cases: unify 'atm' and 'ATM', 'grocery' and 'Grocery'
data['category'] = data.category.str.lower()

In [4]:
# save all raw data to csv
data.to_csv('coinmap_all_raw.csv', index=False)

# 2. Finding missing address information based on geo coordinates

## Geocoordinates decode using 'geopy' package
The same geolocation service (OpenStreetMap) that Coinmap.org uses is chosen for decoding the geo coordinates to location information (address, city, state, postcode and country) <br>
'geopy' python packge (Nominatim class from geopy.geocoders) is used to handle the actual geocoding queries <br>

In [5]:
data['city_geopy'] = np.nan
data['state_geopy'] = np.nan
data['country_geopy'] = np.nan
data['postcode_geopy'] = np.nan
data['address_geopy'] = np.nan

In [6]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from geopy.extra.rate_limiter import RateLimiter
import time

geolocator = Nominatim(user_agent="cryptocurrency") # use geopy.geocoders to make geolocation queries
georeverse = RateLimiter(geolocator.reverse, min_delay_seconds=1) # wait 1 second between each query

def find_location(row):
    """find country, state, city using geopy location queries based on geocoordinates, latitude and longitude"""
    
    coordinates = row.lat, row.lon
 
    # use geolocate query to find the address based on geo coordinates
    try:
        location = georeverse(coordinates,timeout=20)
        
        row['address_geopy'] = location.address
        if 'city' in location.raw['address']:
            row['city_geopy'] = location.raw['address']['city']
        if 'state' in location.raw['address']:
            row['state_geopy'] = location.raw['address']['state']
        if 'country_code' in location.raw['address']:
            row['country_geopy'] = location.raw['address']['country_code']
        if 'postcode' in location.raw['address']:
            row['postcode_geopy'] = location.raw['address']['postcode']
    
    except GeocoderTimedOut as e:
        print("Error: geocode failed to locate the address of coordinates lat:{} lon: {} with message {}".format(lat, lon, e.message))
    
    return row

In [7]:
for n in range(0,14400,200):
    try:
        data.iloc[n:n+200] = data.iloc[n:n+200].apply(find_location, axis=1)
    except:
        print("error occured at range:{}-{}".format(n, n+200))
        pass
    
data.iloc[14400:14448] = data.iloc[14400:14448].apply(find_location, axis=1)

# note: error reported at 13208, unable to find location for 13208
# drop entry 13208, as geopy was not able to decode the address (located on the Ocean) and the country code is not 'US'
data=data.dropna(axis=0, subset=['country_geopy'])

In [8]:
# convert country code to upper case
data['country_geopy'] = data.country_geopy.str.upper()

In [9]:
# save combined data to csv
data.to_csv('coinmap_geopy_combined.csv', index=False)

# 3. Filter to the markets of interet only

All unique categories from Coinmap are: <br>
'atm', 'default', 'grocery', 'food', 'shopping', 'lodging', 'nightlife', 'attraction', 'transport', 'cafe', 'sports','trezor retailer', 'drug store', 'travel agency','educational business'. 

## 3.1 Geodata consistency check between Coinmap and geopy results

In [10]:
# data consistency check

# check country code consistency between coinmap and geopy query results, only focus on results related to US locations
print(data[(data.country != data.country_geopy)&(data.country_geopy == 'US')&(data.country.notnull())].index)
print(data[(data.country != data.country_geopy)&(data.country == 'US')].index)

# one entry 12731 has inconsistent country code of interest: a location in Puerto Rico, which is considered a US territory
# since it's not in the list of interested market, drop the entry
data=data.drop(12731, axis=0)
data = data.reset_index(drop=True)

# entry 6406: a manual lookup by matching phone name leads to: https://www.yelp.com/biz/worlds-smallest-store-new-york
# geocoordinates are wrong, manually update
data.loc[6406, 'city'] = 'New York'
data.loc[6406, 'state'] = 'New York'
data.loc[6406, 'name'] = 'World’s Smallest Store'
data.loc[6406, 'address_geopy'] = '131 Christopher St, New York, NY 10014'
data.loc[6406, 'city_geopy'] = 'New York'
data.loc[6406, 'state_geopy'] = 'New York'
data.loc[6406, 'country_geopy'] = 'US'
data.loc[6406, 'postcode_geopy'] = '10014'
data.loc[6406, 'website'] = 'https://www.yelp.com/biz/worlds-smallest-store-new-york'
data.loc[6406, 'lat'] = 40.7331022
data.loc[6406, 'lon'] = -74.0066992

# entry 8508: a manual lookup by matching phone name leads to: https://www.yelp.com/biz/aloha-crepes-aiea-4?osq=Aloha+Crepes
# geocoordinates are wrong, manually update
data.loc[8508, 'city'] = 'Aiea'
data.loc[8508, 'address_geopy'] = '98-42 Kamehameha Hwy, Aiea, HI 96701'
data.loc[8508, 'city_geopy'] = 'Aiea'
data.loc[8508, 'state_geopy'] = 'Hawaii'
data.loc[8508, 'country_geopy'] = 'US'
data.loc[8508, 'postcode_geopy'] = '96701'
data.loc[8508, 'phone'] = '(808) 483-4448'
data.loc[8508, 'lat'] = 21.3760033
data.loc[8508, 'lon'] = -157.933056

# entry 9394: a manual lookup by matching phone name leads to: https://www.yelp.com/biz/a-and-l-licker-law-firm-st-charles
# geocoordinates are wrong, manually update
data.loc[9394, 'category'] = 'law firm'
data.loc[9394, 'city'] = 'Saint Charles'
data.loc[9394, 'state'] = 'Missouri'
data.loc[9394, 'name'] = 'A & L, Licker Law Firm'
data.loc[9394, 'phone'] = '(636) 916-5400'
data.loc[9394, 'address_geopy'] = '1861 Sherman Dr, St Charles, MO 63303'
data.loc[9394, 'city_geopy'] = 'Saint Charles'
data.loc[9394, 'state_geopy'] = 'Missouri'
data.loc[9394, 'country_geopy'] = 'US'
data.loc[9394, 'postcode_geopy'] = '63303'
data.loc[9394, 'lat'] = 38.7737288
data.loc[9394, 'lon'] = -90.5161671

## 3.2 Filter to US businesses only
As shown below, out of 14445 'worldwide' entries, there are 3580 'US' entries.

In [11]:
# filter to us businesses only
data_us = data[data.country_geopy == 'US'] 
# 3580 'US' entries out of 14445 'worldwide' entries

## 3.3 Filter to the States of interest only
As shown below, out of 3580 US entries, there are 2505 entries from the States of interest.

In [12]:
# filter to the states of interest
stateofinterest = ['California','Massachusetts','Indiana','Ohio','New York','Louisiana','Texas','North Carolina',\
                    'Florida','Maryland','New Jersey','Missouri','Pennsylvania','Michigan','Georgia',\
                   'Illinois','Virginia','Kentucky','Tennessee','Wisconsin','Alabama','Mississippi',\
                   'D.C.','District of Columbia','CA','GA','NC','AL','NY','VA','LA','OH','KY','MO','MS','NJ','MA','TN',\
                  'IN','WI','FL','TX','MD','MI','PA','IL']

data_state = data_us[data_us.state_geopy.isin(stateofinterest)|data_us.state.isin(stateofinterest)]
# 2505 entries for all states of interest out of 3580 US entries

## 3.4 Cross-check and fill in missing city information using 'uszipcode' package

In [13]:
# cross check on zipcode and city using a different geocoding library
from uszipcode import SearchEngine
from uszipcode import Zipcode

data_state['zipcode_uszipcode'] = np.nan
data_state['city_uszipcode'] = np.nan

search = SearchEngine(simple_zipcode=True)
def find_zipcode(row):
    result = search.by_coordinates(row.lat, row.lon, radius=10, returns=1)
    if len(result)>0: 
        result = result[0].to_dict()
        row.zipcode_uszipcode = result['zipcode']
        row.city_uszipcode = result['major_city']
    return row

data_state= data_state.apply(find_zipcode, axis=1)
data_state['city_uszipcode'] = data_state.city_uszipcode.str.lower() # update city to all lower cases

In [14]:
# manually fix the two missing entries for the 'city_uszipcode' and 'zipcode_uszipcode' columns
data_state.loc[6668, 'city_uszipcode'] = 'key west'
data_state.loc[6668, 'zipcode_uszipcode'] = '33040'

data_state.loc[9703, 'city_uszipcode'] = 'mammoth lakes'
data_state.loc[9703, 'zipcode_uszipcode'] = '93546'

In [15]:
# check for city consistency
data_state[(data_state.city_geopy != data_state.city_uszipcode)& data_state.city_geopy.notnull()][['state','state_geopy','city',\
                                                                                    'city_geopy','city_uszipcode',\
                                                                    'postcode_geopy','zipcode_uszipcode']].drop_duplicates()

In [16]:
# check for postcode consistency
data_state[(data_state.postcode_geopy != data_state.zipcode_uszipcode)][['state','state_geopy','city','city_geopy',\
                                                                    'postcode_geopy','zipcode_uszipcode']]

## 3.5 Filter to the cities of interest only
As shown below, out of the 2505 entries for all states of interest, there are 724 entries for the cities of interest. 

In [17]:
# filter to the cities of interest
cityofinterest = {'California':['los angeles','oakland'],
              'Massachusetts':['boston'],
              'Indiana':['indianapolis'],
              'Ohio':['columbus','cleveland','cincinnati'],
              'New York':['new york','new york city','nyc','brooklyn','manhattan','queens','staten island','bronx','the bronx','buffalo'],
              'Louisiana':['new orleans','baton rouge','shreveport'],
              'Texas':['houston','dallas'],
              'North Carolina':['charlotte','raleigh','greensboro','durham'],
              'Florida':['miami','jacksonville'],
              'Maryland':['baltimore'],
              'New Jersey':['newark'],
              'District of Columbia':['washington', 'washington dc'],
              'D.C.':['washington', 'washington dc'],
              'Missouri':['st. louis','kansas city'],
              'Pennsylvania':['philadelphia'],
              'Michigan':['detroit'],
              'Georgia':['atlanta','macon','augusta'],
              'Illinois':['chicago'],
              'Virginia':['norfolk','richmond','richmond city'],
              'Kentucky':['louiville'],
              'Tennessee':['memphis','nashville','nashville-davidson'],
              'Wisconsin':['milwaukee'],
              'Alabama':['birmingham','montgomery','mobile','birmingham, alabama'],
              'Mississippi':['jackson']
             }

# update city names to all lower cases
data_state['city'] = data_state.city.str.lower()
data_state['city_geopy'] = data_state.city_geopy.str.lower()

In [18]:
def match_city(row):
    if (row.state_geopy is not np.nan) and (row.state_geopy in stateofinterest):
        state = row.state_geopy 
        if (row.city_geopy is not np.nan) and (row.city_geopy in cityofinterest[state]):
            return True
        if (row.city is not np.nan) and (row.city in cityofinterest[state]):
            return True
        aif (row.city_uszipcode is not np.nan) and (row.city_uszipcode in cityofinterest[state]):
            return True
    return False

boolean = data_state.apply(match_city, axis=1)
data_city = data_state[boolean]
# 724 entries for cities of interests out of 2505 entries for all states of interest

## 3.6 Filter to the categories of interest based on Coinmap's category label
As shown, out of the 724 entries for the cities of interest, there are at most 698 entries in the categories of interest.

In [19]:
# filter to categories of interest:'atm', 'default','grocery','food','shopping','nightlife', 'cafe','drug store'
data_cat = data_city[data_city.category.isin(['atm', 'default','grocery','food','shopping','nightlife', \
                                                'cafe', 'drug store'])]
# 698 entries of interested categories out of 724 entries for all cities of interest

In [20]:
# check state consistency
data_cat[(data_cat.state != data_cat.state_geopy) & (data_cat.state.notnull())][['state', 'state_geopy']].drop_duplicates()

# update the inconsistent states
data_cat.loc[data_cat.state == 'penna','state'] = 'PA'
data_cat.loc[data_cat.state == 'Nova Iorque','state'] = 'NY'
data_cat.loc[data_cat.state == 'Kalifornien','state'] = 'CA'
data_cat.loc[data_cat.state == 'Нью-Йорк','state'] = 'NY'
data_cat['state'] = data_cat['state'].str.upper()

# deal with the problematic location inconsistency at entry 7954
data_cat.loc[7954, 'city'] = 'houston'
data_cat.loc[7954, 'houseno'] = '17395'
data_cat.loc[7954, 'postcode'] = '77064'
data_cat.loc[7954, 'state'] = 'TEXAS'
data_cat.loc[7954, 'street'] = 'Tomball Pkwy'

In [21]:
# manually fill in the one missing postcode_geopy entry
data_cat.loc[6160, 'postcode_geopy'] = '10019'

In [22]:
# update a few incorrect phone number formats at: 
# 791       786-505-VAPE
# 1698    (786) 564-LOVE
# 11867    1-855-4-BITPAY

data_cat.loc[791, 'phone'] = '786-505-8273'
data_cat.loc[1698, 'phone'] = '786-564-5683'
data_cat.loc[11867, 'phone'] = '855-424-8729'

In [23]:
# save the filtered data to csv
data_cat.to_csv('coinmap_geopy_filtered.csv', index=False)

# 4. Infer business category from business name

In [24]:
data_cat['category_nameinfer'] = np.nan

import re
def infer_category_name(row):
    if row['name'] is not np.nan:
        row['category_nameinfer'] = []
        name = str(row['name']).lower()
        
        categories = [' food','restaurant','cafe','café','parking','cpa','dental','shopping','apparels','church','dentist',\
                      'grill',' bar','attorney',' law ','autopart','auto part','autoservice','auto service',\
                     'boutique','antique','tourism','tourist','printing','advertising','marketplace','market place'\
                     'flower','consulting','store','psychiatrist','design service','designservice',' photo','computer',\
                     'body shop','liquor','sports',' art ','cleaning','repair','tobacco','touring',' spa',' spas ',\
                      'management', 'construction','studio','brew','design',' shop',' tea',' nail','apartment','rental',\
                     ' bbq', ' cajun','storage','seafood','chiropractise','chiropractic','jewelry','software','scooter',\
                     'guitar','musical','counseling','publishing','marketing',' tax','accounting','plumbing', ' jeans',\
                      ' donuts',' tutor',' media',' supply',' hotel',' cater','webservice','web service','podcast',\
                     ' pizza','pizzeria','thai ','sushi bar','sushi',' wings','recording','bistro','chinese','vietnamese',\
                     'dumpling','ham & egg','hot dog','eye care','eyecare','surgery','cookie','jeweler','remodel',\
                     'tatoo',' atm','atm ','atms','financial','auto ',' auto','solar','acupuncture','mexican',' farm',\
                     'farm ','farms','wireless','tavern','wellness',' gym','martial art',' bike','bike ','bikes','coffee',\
                     'bitcoin','lending','real estate','it service','piano','picture','imaging','theatre','motel',\
                     'training','flea market','building','parcel',' deli','deli ','candy story','gas station','beagle',\
                     'coach','cleaner','at&t','dj ',' dj','psychological service','massage','lawyer','hospital',\
                     'laundry',' salon','salon ','bagel','yogurt','kebob','cosmetic','gallery','clothing','dining',\
                     ' wine','italian','tree care','oil change','roofing','fence','fencing','tattoo','diner','painter',\
                      'barber','car testing','collision service','diesel','graphics','flea','bicycle','puppy','auction',\
                     'labs','bitpay','solution','earphone','robotic','leadership','socks','pho ','camera','service',\
                     'technology','3d print',' beauty','beauty ',' health','health ','mover','moving','lawn care']
        for cat in categories:
            if re.search(cat, name) is not None:
                row['category_nameinfer'].append(cat)
        row['category_nameinfer'] = ','.join(row['category_nameinfer'])
        if len(row['category_nameinfer']) == 0:
            row['category_nameinfer'] = np.nan
    return row
            
data_cat = data_cat.apply(infer_category_name, axis=1)

In [25]:
# still 167 entries missing category information
len(data_cat[(data_cat.category_nameinfer.isnull())&(data_cat.category == 'default')])

In [26]:
data_cat = data_cat.reset_index(drop=True)

In [27]:
# save the filtered data to csv
data_cat.to_csv('coinmap_geopy_nameinfer.csv', index=False)

# 5. Cross-reference data from Yelp to fill in business attributes (business category and business phone)

As shown, Yelp API offers several business search services: https://www.yelp.com/developers/documentation/v3/get_started <br>

For the purpose of cross-referencing Coinmap data with Yelp to match business, the relevant services are:
1. business search by phone: https://www.yelp.com/developers/documentation/v3/business_search_phone
2. business search by location and name: https://www.yelp.com/developers/documentation/v3/business_search

In [28]:
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib

In [29]:
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode


# Yelp Fusion no longer uses OAuth as of December 7, 2017.
# You no longer need to provide Client ID to fetch Data
# It now uses private keys to authenticate requests (API Key)
# You can find it on
# https://www.yelp.com/developers/v3/manage_app
API_KEY= mask

# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'

In [30]:
def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    # print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

## 5.1 Matching Yelp records by business phone number

Yelp supports business search by phone: https://www.yelp.com/developers/documentation/v3/business_search_phone,
But the input phone number must be of standard international phone number format: 
It must start with + and include the country code, like +14159083801.<br>

Here, a light python library (phonenumbers python package: 
https://pypi.org/project/phonenumbers/) is used to convert various phone formats to the standard format.<br>

As shown below, 162 entries out of 463 entries with phone number have been matched with Yelp records via phone number.

In [31]:
# update a problematic phone number
data_cat.loc[79,'phone'] = '215-626-7668'

In [32]:
# update phone formats using 'phonenumbers' python package
import phonenumbers
data_cat['phone_formatted'] = np.nan

def format_phone(row):
    if row['phone'] is not np.nan:
        phonenum = phonenumbers.parse(row['phone'],'US')
        row['phone_formatted'] = phonenumbers.format_number(phonenum, phonenumbers.PhoneNumberFormat.E164)
    return row

data_cat = data_cat.apply(format_phone, axis=1)

In [33]:
def yelp_match_by_phone(row):   
    SEARCH_PATH = '/v3/businesses/search/phone'
    
    if row.phone_formatted is not np.nan: 
        data = request(API_HOST, SEARCH_PATH, API_KEY, url_params={'phone': row.phone_formatted})
        if 'error' in data: 
            return np.nan
        else:
            return data
    else:
        return np.nan

In [34]:
yelp_phone = data_cat.apply(yelp_match_by_phone, axis=1)
yelp_phone = pd.DataFrame(yelp_phone)
yelp_phone.columns = ['results']

In [35]:
yelp_phone['total'] = np.nan
yelp_phone['businesses'] = np.nan
yelp_phone['name_yelp'] = np.nan

def unpack_phone(row):
    if row is not np.nan:
        row['total'] = row.results['total']
        row['businesses'] = row.results['businesses']
        row.name_yelp = []
        for i in row['businesses']:
            row['name_yelp'].append(i['name'])
        row['name_yelp'] = ','.join(row['name_yelp'])
    return row

yelp_phone.loc[yelp_phone.results.notnull(), :] = yelp_phone[yelp_phone.results.notnull()].apply(unpack_phone, axis=1)

In [36]:
# check matching results
yelp_phone.total.value_counts()

# 143+15+4 =162 entries have been matched with Yelp records via phone number: 
# note: some businesses have multiple matches from Yelp, further investigation shows that these are typically caused by
# businesses change their name over time, but the phone number, address all remain the same, indicating the same type of business

In [37]:
yelp_phone_matched = yelp_phone[(yelp_phone.total != 0) & yelp_phone.total.notnull()]

# extract category from yelp matched records
yelp_phone_matched['category_yelp']= yelp_phone_matched.apply(lambda x: ','.join(set([j['title'] for i in x.businesses \
                                                                                 for j in i['categories']])), axis=1)

# extract address from yelp matched records
yelp_phone_matched['address_yelp']=yelp_phone_matched.apply(lambda x: ','.join(x.businesses[0]['location']['display_address']),\
                                                                    axis=1)

# update index name
yelp_phone_matched.index.name='id'
yelp_phone_matched = yelp_phone_matched[['name_yelp','category_yelp','address_yelp']]

# join yelp_phone_matched with data_cat
data_cat = data_cat.merge(yelp_phone_matched, how='left', left_index=True, right_index=True)

In [38]:
# save the joined data to csv
data_cat.to_csv('coinmap_geopy_nameinfer_yelpphone.csv', index=False)

In [39]:
# still 130 entries without business category information
len(data_cat[(data_cat.category == 'default')& data_cat.category_nameinfer.isnull() & data_cat.category_yelp.isnull()])

## 5.2 Matching Yelp records by business address & name

business search by location and name: https://www.yelp.com/developers/documentation/v3/business_search <br>
As shown, 21 entries out of the 130 entries missing category information are matched by location and name. 

In [40]:
# extract coinmap address from address-related columns
def combined_address(row):
    addr_cols = ['houseno','street','city','state_geopy', 'postcode']
    address = ','.join([row[i] for i in addr_cols if row[i] is not np.nan])
    if row['houseno'] is not np.nan and row['street'] is not np.nan:
        address = address.replace(',',' ', 1)
    if len(address) > 1:
        return address
    else:
        return np.nan
    
data_cat['coinmap_address'] = data_cat.apply(combined_address, axis=1)

In [41]:
missing = data_cat[(data_cat.category == 'default')& data_cat.category_nameinfer.isnull() & data_cat.category_yelp.isnull()]

In [42]:
def yelp_match_by_address_name(row):   
    SEARCH_PATH = '/v3/businesses/search'
    if row['coinmap_address'] is not np.nan:
        data = request(API_HOST, SEARCH_PATH, API_KEY, url_params={'location':row.coinmap_address,'term':row['name']})
        return data
    else:
        return np.nan

In [43]:
yelp_addressname = missing.apply(yelp_match_by_address_name, axis=1)
yelp_addressname = pd.DataFrame(yelp_addressname)
yelp_addressname.columns = ['results']

In [44]:
yelp_addressname['total'] = np.nan
yelp_addressname['businesses'] = np.nan
yelp_addressname['name_yelp'] = np.nan

def unpack_addressname(row):
    if row.results is not np.nan:
        if 'total' in row.results:
            row['total'] = row.results['total']
        if 'businesses' in row.results:
            row['businesses'] = row.results['businesses']
            row.name_yelp = []
            for i in row['businesses']:
                row['name_yelp'].append(i['name'])
    return row

yelp_addressname = yelp_addressname.apply(unpack_addressname, axis=1)
yelp_addressname = yelp_addressname.loc[yelp_addressname.total >0]

In [45]:
data_cat['phone_yelp_2'] = np.nan
data_cat['category_yelp_2'] = np.nan
data_cat['address_yelp_2'] = np.nan
data_cat['name_yelp_2'] = np.nan

# update based on the match results
index_list = [11, 60, 65, 72, 81, 126, 144, 212, 218, 245, 247, 262, 322, 327, 550, 557]
for idx in index_list:
    data_cat.loc[idx, 'name_yelp_2'] = yelp_addressname.loc[idx,'name_yelp'][0]
    data_cat.loc[idx,'category_yelp_2'] = ','.join(set([i['title'] for i in yelp_addressname.loc[idx, 'businesses'][0]['categories']]))
    data_cat.loc[idx,'address_yelp_2'] = ','.join(yelp_addressname.loc[idx, 'businesses'][0]['location']['display_address'])
    data_cat.loc[idx, 'phone_yelp_2'] = yelp_addressname.loc[idx, 'businesses'][0]['phone']
    
index_list = [165, 177, 284, 313, 491]
for idx in index_list:
    data_cat.loc[idx,'category_yelp_2'] = ','.join(set([i['title'] for i in yelp_addressname.loc[idx, 'businesses']\
                                                      [0]['categories']]))

In [46]:
# still 109 entries missing category information
len(data_cat[(data_cat.category == 'default')& data_cat.category_nameinfer.isnull() & data_cat.category_yelp.isnull() \
        & data_cat.category_yelp_2.isnull()])

## 5.3 Matching Yelp records by business geocoordinates & name
5 entries out of the remaining 109 entries with missing category information are fixed.

In [47]:
def yelp_match_by_coordinate_name(row):   
    SEARCH_PATH = '/v3/businesses/search'
    data = request(API_HOST, SEARCH_PATH, API_KEY, url_params={'latitude':row.lat, 'longitude': row.lon, 'term': row['name']})
    return data

In [48]:
missing = data_cat[(data_cat.category == 'default')& data_cat.category_nameinfer.isnull() & data_cat.category_yelp.isnull() \
         & data_cat.category_yelp_2.isnull()]

In [49]:
yelp_coordinatename = missing.apply(yelp_match_by_coordinate_name, axis=1)
yelp_coordinatename = pd.DataFrame(yelp_coordinatename)
yelp_coordinatename.columns = ['results']

In [50]:
yelp_coordinatename['total'] = np.nan
yelp_coordinatename['businesses'] = np.nan
yelp_coordinatename['name_yelp'] = np.nan

def unpack_coordinatename(row):
    if row.results is not np.nan:
        if 'total' in row.results:
            row['total'] = row.results['total']
        if 'businesses' in row.results:
            row['businesses'] = row.results['businesses']
            row.name_yelp = []
            for i in row['businesses']:
                row['name_yelp'].append(i['name'])
    return row

yelp_coordinatename = yelp_coordinatename.apply(unpack_coordinatename, axis=1)
yelp_coordinatename = yelp_coordinatename[yelp_coordinatename.total>0]

In [51]:
data_cat['category_yelp_3'] = np.nan

# update based on the match results 
index_list = [17]
for idx in index_list:
    data_cat.loc[idx,'category_yelp_3'] = ','.join(set([i['title'] for i in \
                                                        yelp_coordinatename.loc[idx, 'businesses'][0]['categories']]))

data_cat.loc[20,'category_nameinfer'] = 'dating platform'
data_cat.loc[136,'category_nameinfer'] = 'design, cabinetry'
data_cat.loc[445,'category_nameinfer'] = 'gas station'
data_cat.loc[600,'category_nameinfer'] = 'live chat support'

In [52]:
# still 104 entries missing category information
len(data_cat[(data_cat.category == 'default')& data_cat.category_nameinfer.isnull() & data_cat.category_yelp.isnull() \
        & data_cat.category_yelp_2.isnull() & data_cat.category_yelp_3.isnull()])

In [53]:
# save the joined data to csv
data_cat.to_csv('coinmap_geopy_nameinfer_yelpphone_yelplocation.csv', index=False)

# 6. Infer business category using business description from Coinmap.com

29 entries with missing category information are inferred from business description. 

In [54]:
missing = data_cat[(data_cat.category == 'default')& data_cat.category_nameinfer.isnull() & data_cat.category_yelp.isnull() \
        & data_cat.category_yelp_2.isnull() & data_cat.category_yelp_3.isnull()]

In [55]:
data_cat['category_descriptioninfer'] = np.nan

import re
def infer_category_description(row):
    if row['description'] is not np.nan:
        row['category_descriptioninfer'] = []
        desc = str(row['description']).lower()
        
        categories = ['litigation','network','video and animation production','animation video production','blockchain',\
                     'lifestyle advisor','law firm','life coach','web development','telephone company','web design',\
                     'machine shop','photography','marketing','advertising','self-improvement','home improvement',\
                     'homebuilder','support','software','digital content']
        for cat in categories:
            if re.search(cat, desc) is not None:
                row['category_descriptioninfer'].append(cat)
        row['category_descriptioninfer'] = ','.join(row['category_descriptioninfer'])
        
        if len(row['category_descriptioninfer']) == 0:
            row['category_descriptioninfer'] = np.nan
    return row
            
data_cat = data_cat.apply(infer_category_description, axis=1)

In [56]:
# still 75 entries missing category information
len(data_cat[(data_cat.category == 'default')& data_cat.category_nameinfer.isnull() & data_cat.category_yelp.isnull() \
        & data_cat.category_yelp_2.isnull() & data_cat.category_yelp_3.isnull() & data_cat.category_descriptioninfer.isnull()])

In [57]:
# save the joined data to csv
data_cat.to_csv('coinmap_geopy_nameinfer_yelpphone_yelplocation_descriptioninfer.csv', index=False)

# 7. Final cleanup

There is a total of 698 records of interest, among which 79 records are missing business type, 225 records are missing phone number and 117 records are missing website. 

In [59]:
data_cat = pd.read_csv('coinmap_geopy_nameinfer_yelpphone_yelplocation_descriptioninfer.csv', \
                    dtype={'phone_formatted': 'object','phone_yelp_2': 'object', 'zipcode_uszipcode':'object'})

## 7.1 Business name

In [65]:
final = pd.DataFrame()
final['name_coinmap'] = data_cat['name']

def find_yelp_name(row):
    if row['name_yelp'] is not np.nan:
        return row['name_yelp']
    else: 
        return row['name_yelp_2']

final['name_yelp'] = data_cat.apply(find_yelp_name, axis=1)

## 7.2 Business type

In [69]:
def find_business_type(row):
    categories = []
    
    if row['category'] != 'default':
        categories.append(row['category'])
        
    cols = ['category_yelp','category_yelp_2','category_yelp_3','category_nameinfer','category_descriptioninfer']
    for col in cols:
        if row[col] is not np.nan:
            categories.append(row[col])
    if len(categories)>0:
        return ','.join(categories)
    else:
        return np.nan

final['business_type'] = data_cat.apply(find_business_type, axis=1)

## 7.3 Business address

In [70]:
def find_business_address_coinmap(row): 
    if row.city_geopy is not np.nan:
        cols = ['houseno','street','city_geopy','state_geopy', 'postcode_geopy']
    else:
        cols = ['houseno','street','city_uszipcode','state_geopy', 'postcode_geopy']
    address = ','.join([row[i] for i in cols if row[i] is not np.nan])
    
    if row['houseno'] is not np.nan and row['street'] is not np.nan:
        address = address.replace(',',' ', 1)
    
    return address

def find_address_city(row):
    if row['city_geopy'] is not np.nan:
        return row['city_geopy']
    else:
        return row['city_uszipcode']

def find_business_address_yelp(row):
    address = np.nan
    if row['address_yelp'] is not np.nan:
        address = row['address_yelp']
    elif row['address_yelp_2'] is not np.nan:
        address = row['address_yelp_2']
    return address
        
final['address_coinmap'] = data_cat.apply(find_business_address_coinmap, axis=1)  
final['address_yelp'] = data_cat.apply(find_business_address_yelp, axis=1)
final['state'] = data_cat['state_geopy']
final['city'] = data_cat.apply(find_address_city, axis=1)

## 7.4 Business phone number

In [72]:
def find_business_phone(row):
    if row['phone_formatted'] is not np.nan:
        number = row['phone_formatted']
    else:
        number = row['phone_yelp_2']
    
    if number is not np.nan:
        phone = phonenumbers.parse(number,'US')
        number = phonenumbers.format_number(phone, phonenumbers.PhoneNumberFormat.NATIONAL)
        
    return number
        
final['phone'] = data_cat.apply(find_business_phone, axis=1)   

## 7.5 Business website

In [73]:
final['website'] = data_cat['website']

## 7.6 Save as 'csv' file

In [74]:
final = final[['name_coinmap','name_yelp','business_type','state','city','address_coinmap','address_yelp',\
               'phone','website']]
# order the results by state first, then city
final = final.sort_values(['state','city']).reset_index(drop=True)
final.index.name = 'id'

In [728]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 9 columns):
name_coinmap       698 non-null object
name_yelp          178 non-null object
business_type      619 non-null object
state              698 non-null object
city               698 non-null object
address_coinmap    698 non-null object
address_yelp       178 non-null object
phone              473 non-null object
website            581 non-null object
dtypes: object(9)
memory usage: 49.2+ KB


In [729]:
final.head()

,name_coinmap,name_yelp,business_type,state,city,address_coinmap,address_yelp,phone,website
id,,,,,,,,,
0,Eagle's Restaurant,Eagles Restaurant,"food,Soul Food,restaurant",Alabama,birmingham,"2610 16th Street North,birmingham,Alabama,35207","2610 16th St N,Birmingham, AL 35204",(205) 320-0099,http://eaglesrestaurant.menufy.com/
1,Greg's Hot Dogs,NaN,"food,hot dog",Alabama,birmingham,"birmingham,Alabama,35233",NaN,(205) 492-9172,NaN
2,Crestwood Tavern,Crestwood Tavern,"nightlife,Bars,American (Traditional),tavern",Alabama,birmingham,"5500 Crestwood Boulevard,birmingham,Alabama,35212","5500 Crestwood Blvd,Birmingham, AL 35212",(205) 510-0053,https://www.facebook.com/CrestwoodTavernBirmin...
3,Pale Eddie's Pour House,NaN,atm,Alabama,birmingham,"2308 2nd Avenue North,birmingham,Alabama,35203",NaN,(850) 686-5251,http://easybitllc.com
4,Greg's Hot Dogs,NaN,"food,hot dog",Alabama,birmingham,"birmingham,Alabama,35203",NaN,(205) 492-9172,https://www.gregshotdogs.com


In [75]:
# save the final data to csv
final.to_csv('coinmap_business_of_interest.csv')

# note: there is a total of 698 records, among which 79 records are missing business type, 
# 225 records are missing phone number and 117 records are missing website. 